This notebook follows closely the procedure of ```backgroundHists```, but stores the histograms in a different way to showcase the rootpy views system

In [ ]:
from rootpy.io import root_open
from rootpy.tree import TreeChain
from rootpy.plotting import Hist, HistStack, Canvas, views, Legend
import ROOT
from glob import glob
ROOT.gStyle.SetOptStat(0)
ROOT.TH1.AddDirectory(False)

Now, let's create the histogram templates to have consistent binnings across the samples. Rootpy makes very easy to create simple histograms as well as variable-binning ones

In [ ]:
templates = {
    'dijetMass' : Hist(100, 0, 100),
    'cosThetaStar' : Hist([0, 0.2, 0.5, 0.8, 0.9, 1.])
}

Compute the samples weights to be used, for a more detailed description have a look at the ```backgroundHists``` notebook

In [ ]:
from ConfigParser import RawConfigParser
config = RawConfigParser()   
config.optionxform = str       # Last two lines are done because ConfigParser will not preserve case
config.read("hatsConfig.ini")
crossSections = dict([sample, float(xsec)] for sample, xsec in config.items('hatsXsects'))
nProcessed    = dict([sample, int(nPro)] for sample, nPro in config.items('hatsNprocessed'))
weights = {}
luminosity = 1.42    # This is just an example value
for sample in crossSections.keys():
    weights[sample] = luminosity * crossSections[sample]/nProcessed[sample]

In [ ]:
with root_open('histograms.root', 'w') as outfile:
    for sample in weights:
        print 'creating histograms for', sample
        sample_dir = outfile.mkdir(sample)
        sample_dir.cd()
        #get all the root files in the direcotry
        chain = TreeChain(
            'hatsDijets',
            filter(None,popen("xrdfs root://cmseos.fnal.gov/ ls -u /store/user/hats/PyRoot/2017/hatsDijetTrees/hatsTrees_%s/" % sample).read().split('\n'))
        )
        for varname, template in templates.iteritems():
            out_hist = chain.Draw(varname, hist=template.Clone())
            sample_dir.WriteTObject(out_hist, varname)

Many times, we have a ROOT file with a set of histograms that we want to treat consistently (same names, titles, scaling, etc...). Rootpy's views help dealing with it. A view is the extension of a directory/TFile containing histograms on which we want to apply some pre-processing, each view define the type of preprocessing (StyleView, TitleView, SubdirectoryView, SumView, etc...). Views can chained to obtain the desired effect. You can obtain the histogram from any view by calling the ```Get``` method, like on a TFile.

In [ ]:
histo_file = root_open('histograms.root')
QCD_HT500to700 = views.StyleView(
    views.TitleView(
        views.ScaleView(
            histo_file.QCD_HT500to700, #input directory
            weights['QCD_HT500to700']
        ),
        'QCD_HT500to700' #the title
    ),
    fillstyle = 'solid',
    drawstyle = 'hist', #how do we want to draw it, by default
    fillcolor = '#0067a9',
    inlegend = True, #in case of a stack histo, do we want it in the Legend?
    legendstyle = 'f', #how do we want the legend marker? (f,l,p)
    linecolor = 'black',
    linewidth = 1
    #markerstyle ...
    #markercolor ...
)

Now, each histogram will have the same pre-processing

In [ ]:
canvas = Canvas(800, 800)
htemp = QCD_HT500to700.Get('dijetMass')
htemp.Draw()
canvas.Draw()

In [ ]:
htemp = QCD_HT500to700.Get('cosThetaStar')
htemp.Draw()
canvas.Draw()

Now, complete the exercise making a set of stack plots, either using rootpy's HistStack or views.StackView